In [2]:
!pip install torch torchvision transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import pandas as pd

In [4]:
dataset = pd.read_csv('news_dataset.csv')

In [5]:
dataset.head()

,text,labels
0,charikar afghanistan reuters afghan autho...,1
1,new york reuters britain the united state...,1
2,vatican city reuters pope francis canonize...,1
3,yes this will happen america needs a wall at...,0
4,there s a lot of buzz about the gop s latest e...,0


In [6]:
dataset.shape

(44878, 2)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44878 entries, 0 to 44877
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44878 non-null  object
 1   labels  44878 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.3+ KB


In [8]:
dataset.value_counts('labels')

,count
labels,
0,23471
1,21407


In [9]:
dataset.dropna(inplace=True)

In [10]:
dataset.head(10)

,text,labels
0,charikar afghanistan reuters afghan autho...,1
1,new york reuters britain the united state...,1
2,vatican city reuters pope francis canonize...,1
3,yes this will happen america needs a wall at...,0
4,there s a lot of buzz about the gop s latest e...,0
5,geneva reuters the united nations has veri...,1
6,century wire says three weeks ago while the ...,0
7,republican presidential nominee donald trump h...,0
8,brussels reuters turkish foreign minister ...,1
9,in discussing shootings and crime in the first...,0


In [11]:
dataset.shape

(44878, 2)

In [12]:
df = dataset

In [13]:
df.head(10)

,text,labels
0,charikar afghanistan reuters afghan autho...,1
1,new york reuters britain the united state...,1
2,vatican city reuters pope francis canonize...,1
3,yes this will happen america needs a wall at...,0
4,there s a lot of buzz about the gop s latest e...,0
5,geneva reuters the united nations has veri...,1
6,century wire says three weeks ago while the ...,0
7,republican presidential nominee donald trump h...,0
8,brussels reuters turkish foreign minister ...,1
9,in discussing shootings and crime in the first...,0


# Split dataset into Train, Val and Test sets



In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# train-temp split (80:20)

train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)

# validation-test split (10:10) -> 80:10:10

val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [16]:
train_data.shape

(35902, 2)

In [17]:
train_data.head(10)

,text,labels
27091,cairo reuters egyptian security forces kil...,1
28922,donald trump is a liar not the kind of lies t...,0
35536,obama has a knack for going overseas and dispa...,0
16621,washington it sits on the corsica river alon...,0
33,join patrick every wednesday at independent ta...,0
29646,why couldn t the parents be notified of his de...,0
5233,in a group called the christian movement fo...,0
20347,it is so great to be back in florida my secon...,0
1550,tokyo reuters russia s military chief warn...,1
1888,vladivostok russia reuters russian presid...,1


In [18]:
train_data.reset_index(inplace = True)

In [19]:
train_data = train_data.drop(columns=['index'])

In [20]:
train_data.head(10)

,text,labels
0,cairo reuters egyptian security forces kil...,1
1,donald trump is a liar not the kind of lies t...,0
2,obama has a knack for going overseas and dispa...,0
3,washington it sits on the corsica river alon...,0
4,join patrick every wednesday at independent ta...,0
5,why couldn t the parents be notified of his de...,0
6,in a group called the christian movement fo...,0
7,it is so great to be back in florida my secon...,0
8,tokyo reuters russia s military chief warn...,1
9,vladivostok russia reuters russian presid...,1


In [21]:
train_data.value_counts('labels')

,count
labels,
0,18745
1,17157


In [22]:
train_data.shape

(35902, 2)

In [23]:
train_data.columns

Index(['text', 'labels'], dtype='object')

In [24]:
train_data.to_csv('train_data.csv', index=False)

In [25]:
val_data.shape

(4488, 2)

In [26]:
val_data.head(10)

,text,labels
2528,jakarta reuters indonesian counter terrori...,1
20631,london reuters london mayor sadiq khan sai...,1
200,when donald trump named steve bannon as his ch...,0
37304,uh oh billionaire green guru tom stayer says ...,0
4704,rep trey gowdy was full of sarcasm rightfull...,0
11742,new york reuters authorities were investig...,1
34598,just one more reminder of why president trump ...,0
11547,oklahoma city reuters oklahoma lawmakers w...,1
25776,convincing president trump to release his tax ...,0
26139,hillary is without a doubt the worst and most...,0


In [27]:
val_data.reset_index(inplace = True)

In [28]:
val_data.head(10)

,index,text,labels
0,2528,jakarta reuters indonesian counter terrori...,1
1,20631,london reuters london mayor sadiq khan sai...,1
2,200,when donald trump named steve bannon as his ch...,0
3,37304,uh oh billionaire green guru tom stayer says ...,0
4,4704,rep trey gowdy was full of sarcasm rightfull...,0
5,11742,new york reuters authorities were investig...,1
6,34598,just one more reminder of why president trump ...,0
7,11547,oklahoma city reuters oklahoma lawmakers w...,1
8,25776,convincing president trump to release his tax ...,0
9,26139,hillary is without a doubt the worst and most...,0


In [29]:
val_data = val_data.drop(columns=['index'])

In [30]:
val_data.head(10)

,text,labels
0,jakarta reuters indonesian counter terrori...,1
1,london reuters london mayor sadiq khan sai...,1
2,when donald trump named steve bannon as his ch...,0
3,uh oh billionaire green guru tom stayer says ...,0
4,rep trey gowdy was full of sarcasm rightfull...,0
5,new york reuters authorities were investig...,1
6,just one more reminder of why president trump ...,0
7,oklahoma city reuters oklahoma lawmakers w...,1
8,convincing president trump to release his tax ...,0
9,hillary is without a doubt the worst and most...,0


In [31]:
val_data.columns

Index(['text', 'labels'], dtype='object')

In [32]:
val_data.to_csv('val_data.csv', index=False)

In [33]:
val_data.shape

(4488, 2)

In [34]:
test_data.shape

(4488, 2)

In [46]:
test_data.to_csv('test_data.csv', index=False)

In [36]:
test_data.head(10)

,text,labels
1911,london reuters lawmakers from britain s ru...,1
39733,on friday more than half the nation will mour...,0
23134,nothing says thanksgiving or family like a...,0
32586,century wire asks will this be the beginning ...,0
9301,florence italy reuters prime minister the...,1
29183,chilpancingo mexico reuters a protest by ...,1
3905,shawn helton century wirethis week a house ju...,0
38017,washington reuters the president of micros...,1
24738,beijing washington reuters u s president ...,1
11562,in case you haven t heard eric trump recently...,0


In [37]:
test_data.reset_index(inplace = True)

In [38]:
test_data = test_data.drop(columns=['index'])

In [39]:
test_data.head(10)

,text,labels
0,london reuters lawmakers from britain s ru...,1
1,on friday more than half the nation will mour...,0
2,nothing says thanksgiving or family like a...,0
3,century wire asks will this be the beginning ...,0
4,florence italy reuters prime minister the...,1
5,chilpancingo mexico reuters a protest by ...,1
6,shawn helton century wirethis week a house ju...,0
7,washington reuters the president of micros...,1
8,beijing washington reuters u s president ...,1
9,in case you haven t heard eric trump recently...,0


In [41]:
test_data.shape

(4488, 2)

In [42]:
train_data.head()

,text,labels
0,cairo reuters egyptian security forces kil...,1
1,donald trump is a liar not the kind of lies t...,0
2,obama has a knack for going overseas and dispa...,0
3,washington it sits on the corsica river alon...,0
4,join patrick every wednesday at independent ta...,0


In [43]:
train_data.columns

Index(['text', 'labels'], dtype='object')

In [44]:
val_data.columns

Index(['text', 'labels'], dtype='object')

In [45]:
val_data.head()

,text,labels
0,jakarta reuters indonesian counter terrori...,1
1,london reuters london mayor sadiq khan sai...,1
2,when donald trump named steve bannon as his ch...,0
3,uh oh billionaire green guru tom stayer says ...,0
4,rep trey gowdy was full of sarcasm rightfull...,0


In [48]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Tokenize the dataset

In [49]:
from transformers import DistilBertTokenizer

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [50]:
# Tokenization function
"""this function, tokenizes the text column of the dataset and returns a dictionary with tokenized values."""

def tokenize_data(data, max_len=128):
    encodings = tokenizer(
        data['text'].tolist(),
        add_special_tokens=True,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    labels = torch.tensor(data['labels'].tolist(), dtype=torch.long)
    return encodings, labels

In [51]:
from torch.utils.data import Dataset

""" This class takes pre-tokenized data (encodings) and labels, then returns individual samples """

class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # Tokenized input_ids & attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return {
            'input_ids': self.encodings['input_ids'][index],
            'attention_mask': self.encodings['attention_mask'][index],
            'labels': self.labels[index]
        }

In [52]:
# Tokenize train_data and val_data

train_encodings, train_labels = tokenize_data(train_data, max_len=128)
val_encodings, val_labels = tokenize_data(val_data, max_len=128)
test_encodings, test_labels = tokenize_data(test_data, max_len=128)

In [54]:
print(train_encodings)

{'input_ids': tensor([[  101, 11096, 26665,  ...,  5368,  2295,   102],
        [  101,  6221,  8398,  ...,  2000,  2032,   102],
        [  101,  8112,  2038,  ..., 16558, 12384,   102],
        ...,
        [  101,  2096,  2009,  ...,  2102,  2008,   102],
        [  101,  2043,  1996,  ...,  1998,  7040,   102],
        [  101,  2899, 26665,  ...,  2491,  3519,   102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [55]:
print(train_labels)

tensor([1, 0, 0,  ..., 0, 0, 1])


# Create Dataset Objects

In [56]:

train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)
test_dataset = FakeNewsDataset(test_encodings, test_labels)

In [66]:
print(train_encodings['input_ids'][0])
print(train_encodings['attention_mask'][0])
print(train_labels[0])

tensor([  101, 11096, 26665,  6811,  3036,  2749,  2730,  2274,  6878, 17671,
         1999,  1037,  8118,  2006,  6928,  2006,  1037,  5532,  3295,  2108,
         2109,  2011,  1996,  2177,  2005,  2731,  1999,  1996, 15179,  7160,
         2874,  1997, 21146,  2099, 14702,  3148,  2167,  1997, 11096,  1996,
         4592,  3757,  2056,  3036,  2749,  4727,  2416,  2060,  2111,  1998,
         8243,  9290,  1998,  4255,  1999,  1996,  3169,  1037,  3757,  4861,
         2056,  1996,  4861,  2106,  2025,  4957,  1996, 17671,  2000,  2151,
         3563,  2177,  2144,  6811,  3036,  2749,  2031, 19787,  2019,  5499,
         2110,  8727,  1999,  1996,  3701,  5532,  2555,  1997,  2167, 20837,
         2073, 17671,  2031,  2730,  5606,  1997,  2610,  1998,  3548,  2197,
         3204,  5279,  2387,  2049,  2757, 21292,  2886,  1999,  2049,  2715,
         2381,  2043, 17671,  2730,  2062,  2084,  2012,  1037,  2167, 20837,
         8806,  2053,  2177,  2038,  3555,  5368,  2295,   102])

In [67]:
print(train_dataset[0])

{'input_ids': tensor([  101, 11096, 26665,  6811,  3036,  2749,  2730,  2274,  6878, 17671,
         1999,  1037,  8118,  2006,  6928,  2006,  1037,  5532,  3295,  2108,
         2109,  2011,  1996,  2177,  2005,  2731,  1999,  1996, 15179,  7160,
         2874,  1997, 21146,  2099, 14702,  3148,  2167,  1997, 11096,  1996,
         4592,  3757,  2056,  3036,  2749,  4727,  2416,  2060,  2111,  1998,
         8243,  9290,  1998,  4255,  1999,  1996,  3169,  1037,  3757,  4861,
         2056,  1996,  4861,  2106,  2025,  4957,  1996, 17671,  2000,  2151,
         3563,  2177,  2144,  6811,  3036,  2749,  2031, 19787,  2019,  5499,
         2110,  8727,  1999,  1996,  3701,  5532,  2555,  1997,  2167, 20837,
         2073, 17671,  2031,  2730,  5606,  1997,  2610,  1998,  3548,  2197,
         3204,  5279,  2387,  2049,  2757, 21292,  2886,  1999,  2049,  2715,
         2381,  2043, 17671,  2730,  2062,  2084,  2012,  1037,  2167, 20837,
         8806,  2053,  2177,  2038,  3555,  5368, 

# Load the model

Note: The model is by default in evaluation mode model.eval(), so we need to execute model.train() for fine tuning.

In [57]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


**Training arguments**

In [59]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",           # Directory for saving results
    eval_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-5,              # Initial learning rate
    per_device_train_batch_size=16,  # Batch size per GPU
    num_train_epochs=3,              # Number of epochs
    warmup_steps=600,                # Number of warmup steps
    weight_decay=0.01,               # Regularization
    logging_dir="./logs",            # Directory for logs
    logging_steps=10                 # Log every 10 steps
)

# Fine-tuning with Trainer

In [60]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aqila (aqila-farahmand) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.051000,0.012960
2,0.089900,0.011552
3,0.000100,0.004178


TrainOutput(global_step=6732, training_loss=0.01910065761882844, metrics={'train_runtime': 2142.0681, 'train_samples_per_second': 50.281, 'train_steps_per_second': 3.143, 'total_flos': 3566883409910784.0, 'train_loss': 0.01910065761882844, 'epoch': 3.0})

# Save the model

In [65]:
trainer.model.save_pretrained("./fake_news_detection_model")
tokenizer.save_pretrained('./fake_news_detection_model')

('./fake_news_model/tokenizer_config.json',
 './fake_news_model/special_tokens_map.json',
 './fake_news_model/vocab.txt',
 './fake_news_model/added_tokens.json')